# <center> *PROJECT-3. EDE + Feature Engineering*
# <center> *Соревнования на Kaggle*

<span style="color:#59afe1"> 

## Содержание:

+ [Постановка задачи](#p1)
+ [Подготовка Jupyter Notebook](#p2)
+ [Загрузка данных](#p3)
+ [Предварительный анализ данных](#p4)
+ [Разведывательный анализ и обработка данных](#p5)

    + [Проанализируем корреляционную связь числовых признаков с целевым признаком *reviewer_score*](#p51)
    + [Обработка признака *hotel_address*](#p52)
    + [На основе признаков *country* и reviewer_nationality создадим признак *resident*](#p53)
    + [Создадим признак percentage_of_reviews](#p54)
    + [Обработка признака *review_date*](#p55)
    + [Обработка признака *hotel_name*](#p56)
    + [Обработка признака *reviewer_nationality*](#p57)
    + [Обработка признаков *positive_review* и *negative_review*](#p58)
    + [Обработка признака *tags*](#p59)
    + [Обработка признака *lat* и *lng* ](#p510)
    + [Выбор признаков для обучения модели](#p511)
    
+ [Проверка обучающих признаков](#p6)
+ [Подготовка тестовых данных](#p7)

    + [загрузка данных](#p71)
    + [создадим признак *country_cod*](#p72)
    + [создадим признак *resident*](#p73)
    + [создадим признак *good_words*](#p74)
    + [создадим признак *nationality_cod*](#p75)
    + [создадим признаки *friendly_words* и *no_friendly_words*](#p76)
    + [создадим признак *percentage_positive_reviews*](#p77)
    + [создадим признак *type_of_trip*](#p78)
    + [создадим признак *view*](#p79)
    + [создадим признак *type_of_premium_room*](#p710)
    + [Результирующие test_data](#p711)

+ [Машинное обучение](#p8)

<span style="color:#59afe1"> 

## ПОСТАНОВКА ЗАДАЧИ <a id='p1'>

<span style="color:#59afe1"> 

Кейс:

Представьте, что вы работаете дата-сайентистом в компании Booking. Одна из проблем компании — это нечестные отели,  
которые накручивают себе рейтинг. Одним из способов обнаружения таких отелей является построение модели, которая   
предсказывает рейтинг отеля. Если предсказания модели сильно отличаются от фактического результата, то, возможно,   
отель ведёт себя нечестно, и его стоит проверить.

Вам поставлена задача создать такую модель.

Исходные данные:

В этом модуле мы будем работать с датасетом, в котором содержатся сведения о 515 000 отзывов на отели Европы.  
Первоначальная версия датасета содержит 17 полей со следующей информацией:

- hotel_address — адрес отеля;
- review_date — дата, когда рецензент разместил соответствующий отзыв;
- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;
- hotel_name — название отеля;
- reviewer_nationality — страна рецензента;
- negative_review — отрицательный отзыв, который рецензент дал отелю;
- review_total_negative_word_counts — общее количество слов в отрицательном отзыве;
- positive_review — положительный отзыв, который рецензент дал отелю;
- review_total_positive_word_counts — общее количество слов в положительном отзыве;
- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;
- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;
- total_number_of_reviews — общее количество действительных отзывов об отеле;
- tags — теги, которые рецензент дал отелю;
- days_since_review — количество дней между датой проверки и датой очистки;
- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там дополнительных оценок без отзывов.
- lat — географическая широта отеля;
- lng — географическая долгота отеля.

Источник датасета: [hotel.csv](https://drive.google.com/file/d/1Qj0iYEbD64eVAaaBylJeIi3qvMzxf2C_/view?usp=sharing)

<span style="color:#59afe1"> 

## ПОДГОТОВКА JUPYTER NOTEBOOK <a id='p2'>

In [ ]:
# Импорт библиотеки для работы с данными
import pandas as pd
import numpy as np

# импортируем библиотеку для работы с кодировщиками
import category_encoders as ce 

# импортируем библиотеку для работы с подсчетом в списках
from collections import Counter

# импорт библиотеки для работы со статистическим анализом
from scipy import stats
import statsmodels.api as sm
from statsmodels import stats as sms

# импорт библиотеки для регулярных выражений
import re

# библиотека для работы с геоданными
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
# чтобы убрать проверку сертификатов
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

# Загружаем специальный инструмент для разбивки выборок из данных:  
from sklearn.model_selection import train_test_split

# Загружаем библиотеки для создания модели:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 

# импорт библиотек для работы с plotly графикой
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.express as px
from plotly.subplots import make_subplots

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# импорт библиотеки для игнорирования сообщений warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# фиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip3 freeze > requirements.txt

<span style="color:#59afe1"> 

## ЗАГРУЗКА ДАННЫХ <a id='p3'>

In [ ]:
hotels =pd.read_csv('data/hotels.csv')
hotels.head(5)

<span style="color:#59afe1"> 

## ПРЕДВАРИТЕЛЬНЫЙ АНАЛИЗ ДАННЫХ <a id='p4'>

In [ ]:
# информация о типам данных в таблице и нулевых значениях
hotels.info()

In [ ]:
# проанализируем основные статистические показатели числовых признаков
hotels.describe()

In [ ]:
# проанализируем основные статистические показатели не числовых признаков
hotels.describe(exclude=['int64','float64'])

In [ ]:
# проверим на наличие полных дубликатов
print('Количество дубликатов: {}'.format(hotels[hotels.duplicated()].shape[0]))
print('Размер таблицы до удаления дубликатов: {}'.format(hotels.shape))
# Удаляем дубликаты:
df = hotels.drop_duplicates()
print('Размер таблицы после удаления дубликатов: {}'.format(df.shape))

<span style="color:#59afe1"> 

Поставим своей задачей извлечь из признаков новые признаки, влияющие на reviewer_score для отеля. 

- проанализируем корреляционную связь числовых признаков с целевым.

- hotel_address.  
    Из этого признака можно извлечь информацию о стране и городе в котором находится отель.  
    По полученной информации можно получить ТОП "лучших" стран, с точки зрения  
    оценки рецензента. После создадим числовой признак - место страны в ТОПе.  
    Используя признак reviewer_nationality, можно создать ,бинарный признак "resident".  
    Возможную зависимость, оценки рецензента от полученного признака можно оценить в дальнейшем.  

- Посмотрим на зависимость reviewer_score от соотношения  
    percentage_of_reviews = total_number_of_reviews/(total_number_of_reviews+additional_number_of_scoring).    
    Какой процент посетителей из тех, что поставили оценку оставляли свой отзыв об отеле.
    
- review_date.  
    Приведем этот признак к типу данных datatime. Извлечем год и месяц в котором была оставлена оценка.  
    Проанализируем зависимость reviewer_score от месяца и года в котором оставляли отзыв об отеле.

- hotel_name.  
    Я думаю, из этого признака можно извлечь "красивые"/"плохие" слова, составить из них ТОП и сделать  
    бинарные признаки - присутствует/отсутствует в название отеля "красивое"/"плохое" слово.     
    Я имею ввиду, что если в имени отеля есть слова  "Legend", "Mercure", "International" и т.д.,   
    скоре всего эти имена в какой то степени отражают уровень отеля. Или на крайний случай, отношение   
    посетителя к отелю, что так же влияет не его оценку.

- reviewer_nationality.  
    Проанализировать влияние признака на оценку посетителя. При наличие связи выявить ТОП   
    "хороших"/"плохих" национальностей и сделать соответствующие категориальные признаки.  

- negative_review.  
    Проанализировать на ТОП "плохих" слов. Сделать бинарный признак.  
    Найдем количество негативных отзывов для каждого отеля.

- positive_review.  
    Проанализировать на ТОП "хороших" слов. Сделать бинарный признак.
    Найдем количество позитивных отзывов для каждого отеля.  

- lat, lng.  
    Используем признаки, чтобы с помощью дополнительных источников узнать расстояния от отеля до метро, кофейных, центра города. 
    При создание новых признаков учтем, что присутствуют пропуски значений. 

In [ ]:
# создадим список в который будет добавлять признаки, подходящие под обучение модели 
list_features =[]

<span style="color:#59afe1"> 

## РАЗВЕДЫВАТЕЛЬНЫЙ АНАЛИЗ И ОБРАБОТКА ДАННЫХ <a id='p5'>

<span style="color:#59afe1"> 

### Проанализируем корреляционную связь числовых признаков с целевым признаком *reviewer_score* <a id='p51'>

In [ ]:
# соберем матрицу корреляции из числовых признаков
df_corr_rs = hotels.corr(numeric_only=True)

In [ ]:
# представим матрицу корреляции на тепловой карте
# так как названия признаков громоздкие "спрячем" их под номера, а сама информация будет появляться при наведение на ячейку
# для этого создадим матрицу признаков отображаемых при наведение
# матрица будет иметь размерность LxL, где L-это количество признаков в матрице
# соберем список числовых признаков
list_f =list(df_corr_rs.index)
# заполним матрицу списками со всеми возможными пересечениями признаков
matrix_f = []
for i in range(0,len(list_f)):
    new_list = []                                   # на каждой итерации создаем пустой список
    for j in range(0,len(list_f)):                  # создаем второй цикл для заполнения новых списков двумя признаками
        new_list.append([list_f[i],list_f[j]])      # заполняем список двумя признаками
    matrix_f.append(new_list)                       # после выход их второго цикла передаем список в исходную матрицу


fig_1 = go.Figure(data=go.Heatmap(
                   z = df_corr_rs.round(3),
                   zmin=-1,
                   zmax=1,
                   zmid=0,
                   texttemplate='%{z}',
                   meta=matrix_f,
                   hovertemplate="%{meta[0]}<br>%{meta[1]}<extra></extra>",
                   colorscale =[
                       [0.0,'red'],
                       [0.05,'red'],
                       [0.05,'orange'],
                       [0.15,'orange'],
                       [0.15,'yellow'],
                       [0.25,'yellow'],
                       [0.25,'green'],
                       [0.35,'green'],
                       [0.35,'Aqua'],
                       [0.45,'Aqua'],
                       [0.45,'white'],
                       [0.5,'white'],
                       [0.55,'white'],
                       [0.55,'Aqua'],
                       [0.65,'Aqua'],
                       [0.65,'green'],
                       [0.75,'green'],
                       [0.75,'yellow'],
                       [0.85,'yellow'],
                       [0.85,'orange'],
                       [0.95,'orange'],
                       [0.95,'red'],
                       [1.0,'red']
                   ]
                   ))
# настроим оформление рабочей поверхности
fig_1.update_layout(
    title ={
        'text':'Матрица корреляции', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_1.show()

<span style="color:#59afe1">

**Вывод**

Корреляционный анализ показал:
- целевой признак не имеет корреляционной связи с:
    - additional_number_of_scoring;
    - total_number_of_reviews;
    - total_number_of_reviews_reviewer_has_given;
    - lat;
    - lat.
- целевой признак имеет очень слабую положительную корреляционную связь с:
    - review_total_positive_word_counts.
- целевой признак имеет слабую положительную корреляционную связь с:
    - average_score.
- целевой признак имеет слабую отрицательную корреляционную связь с:
    - review_total_negative_word_counts.

Признаки не имеющие корреляционной связи, не попадут в список для обучения модели.

Помимо этого признаки additional_number_of_scoring и total_number_of_reviews имеют сильную корреляционную связь.  
Получим из них признак percentage_of_reviews и удалим оба.

In [ ]:
# добавим выбранные признаки в список list_features
list_features.extend(['review_total_positive_word_counts','average_score','review_total_negative_word_counts'])

<span style="color:#59afe1"> 

### Обработка признака *hotel_address* <a id='p52'>

In [ ]:
# взглянем на структуру адреса
hotels['hotel_address'].value_counts().iloc[100:120]

1. Если отель не из United Kingdom, то структура адреса : ".... город страна"
2. Если отель из United Kingdom, то структура адреса : ".... город United Kingdom"

In [ ]:
# взглянем на значения полученные извлечением последнего слова в строке
hotels['hotel_address'].str.findall('\w+').str.get(-1).value_counts()

In [ ]:
# взглянем на значения полученные извлечением предпоследнего слова в строке
hotels['hotel_address'].str.findall('\w+').str.get(-2).value_counts()

Очевидно, что нет ошибки в предположение: информация о стране содержится в последнем слове строки адреса.  
Видно, что в данных представлены отели только из одного города в стране. Если это Italy, то город обязательно Milan.  
Поэтому, для дальнейшего анализа нет смысла извлекать и страну и город.

In [ ]:
# создадим признак страны отеля
hotels['country']= hotels['hotel_address'].str.findall('\w+').str.get(-1)
# заменим значение Kingdom на United Kingdom
hotels['country']=hotels['country'].apply(lambda x: 'United Kingdom' if x == 'Kingdom' else x)
hotels['country'].value_counts()

In [ ]:
# создадим признак города отеля
hotels['city']= hotels['hotel_address'].str.findall('\w+').str.get(-2)
# заменим значение United на London
hotels['city']=hotels['city'].apply(lambda x: 'London' if x == 'United' else x)
hotels['city'].value_counts()

Проанализируем распределение оценок рецензента в зависимости от страны отеля

In [ ]:
fig_country = go.Figure()

# настроим оформление рабочей поверхности
fig_country.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от страны', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_country.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_country.update_yaxes(
    categoryorder ='mean ascending',
)

# Добавим диаграммы на рабочую поверхность
for mask in sorted(hotels['country'].unique()):
    fig_country.add_trace(go.Box(
        y=hotels[hotels['country'] ==mask]['country'],
        x=hotels[hotels['country'] ==mask]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=str(mask)
        )
                    )

# расположим коробчатые графики горизонтально
fig_country.update_traces(
    orientation='h'
)
fig_country.show()

Из анализа графика можно сказать, что оценки отеля отличаются в зависимости от страны отеля, но эти отличия не выгладят статистически значимыми.  
Проведем для представленных распределений статистический тест.

In [ ]:
# задаём уровень значимости
alpha = 0.05 
print("Уровень значимости alpha = {:.2f}".format(alpha))

In [ ]:
# функция для принятия решения о нормальности
def decision_normality(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Распределение отлично от нормального'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. Распределение является нормальным'.format(alpha))


# функция для принятия решения об отклонении нулевой гипотезы
def decision_hypothesis(p):
    print('p-value = {:.3f}'.format(p))
    if p <= alpha:
        print('p-значение меньше, чем заданный уровень значимости {:.2f}. Отвергаем нулевую гипотезу в пользу альтернативной.'.format(alpha))
    else:
        print('p-значение больше, чем заданный уровень значимости {:.2f}. У нас нет оснований отвергнуть нулевую гипотезу.'.format(alpha))

Оценка рецензента зависит от страны отеля?

**Сформулируем нулевую и альтернативные гипотезы**

*Нулевая гипотеза*: Оценка рецензента не зависит от страны отеля:

$$ H_0 : μ_1 = μ_2 = μ_3 = μ_4 = μ_5$$

*Альтернативная гипотеза*: Оценка рецензента зависит от страны отеля.

$$ H_1 : μ_1 \neq μ_2 \neq μ_3 \neq μ_4 \neq μ_5$$



**Проверка на нормальность**

С помощью теста Шапиро-Уилка проверим, распределён ли признак нормально.

In [ ]:
# для сокращения длины кода осуществим проверку с помощью цикла для этого:
# проводим тест Шапиро-Уилка
for mask in hotels['country'].unique():
    print('Распределение оценок рецензентов для отелей из',str(mask)+':')
    result = stats.shapiro(hotels[hotels['country'] ==mask]['reviewer_score'])
    decision_normality(result[1])
    print()
    
print('Количество сравниваемых групп:',len(hotels['country'].unique()))

**Выберем подходящий статистический тест**

Для выбора нужного теста воспользуемся алгоритмом выбора теста. Для этого ответим на следующие вопросы:
* Какой тип у исследуемого признака? — Количественный.
* Сколько сравниваемых групп? — шесть.
* Группы зависимы? — Нет.
* Признак распределён по нормальному закону? — Нет. 

Для проверки нашей гипотезы можно использовать критерий Краскела — Уоллиса.

**Проведём тест**

In [ ]:
# соберем список из тестируемых зависимостей
list_df_countries =[]
# воспользуемся списком list_countries, полученного на этапе ранее
for mask in hotels['country'].unique():
    list_df_countries.append(hotels[hotels['country']==mask]['reviewer_score'])
    # проводим тест
_, p = stats.kruskal(*list_df_countries)
decision_hypothesis(p)

<span style="color:#59afe1">

**Вывод**

Статистический тест показал наличие связи между страной отеля и оценкой рецензента.   
Очевидно, что мы оставляем данный признак для обучения нашей модели.   
Для этого кодируем наш признак с помощью порядкового кодирования.  
В нашем кодирование номер будет означать более высокие места в ТОП стран по среднему значению оценки рецензента.  
Первое место имеет самое большое значение кода.

In [ ]:
# составим список ТОП стран отеля по среднему значению оценки рецензента
list_top_countries = list(hotels.groupby('country')['reviewer_score'].mean().sort_values().index)

# составим карту кодирования (Словарь со значениями)
dict_map_c ={}
for i in range(0,len(list_top_countries)):
    dict_map_c[list_top_countries[i]]=i+1

# создаем объект OrdinalEncoder, col - имя столбца, mapping - словарь с описанием кодировки
ord_encoder_c = ce.OrdinalEncoder(mapping=[{
	'col': 'country',
	'mapping': dict_map_c
}])
# применяем трансформацию к столбцу
hotels['country_cod'] = ord_encoder_c.fit_transform(hotels[['country']])
#  проверим корректность замены
print(hotels['country'].value_counts(),hotels['country_cod'].value_counts())

In [ ]:
# добавим признак country_cod в список list_features
list_features.append('country_cod')

<span style="color:#59afe1"> 

### На основе признаков *country* и reviewer_nationality создадим признак *resident* <a id='p53'>

In [ ]:
# значение в признаки 1 - посетитель резидент страны
# значение признака 0 - посетитель не резидент страны
hotels['resident'] = hotels.apply(lambda x: 1 if (x['country']==x['reviewer_nationality'][1:-1]) else 0,axis=1)
# проверим фрагмент таблицы
hotels[['country','reviewer_nationality','resident']].head(10)

проанализируем связь полученного признака и *reviewer_score*

In [ ]:
fig_3 = go.Figure()
# пропишем условия выбора данных
mask_r_1 = hotels['resident'] == 1
mask_r_0 = hotels['resident'] == 0

# сформируем список условий и имен
list_resident = [[mask_r_1,'Resident'],[mask_r_0,'No Resident']]

# настроим оформление рабочей поверхности
fig_3.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от его статуса в стране', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_3.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_3.update_yaxes(
    categoryorder ='mean ascending',
)

# Добавим диаграммы на рабочую поверхность
for mask in list_resident:
    fig_3.add_trace(go.Box(
        y=hotels[mask[0]]['resident'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_3.update_traces(
    orientation='h'
)
fig_3.show()


Из анализа графика можно сказать, что оценки отеля отличаются в зависимости от статуса рецензента, но эти отличия не выгладят статистически значимыми.  
Проведем для представленных распределений статистический тест.

Как соотносятся оценки рецензентов в зависимости от их статуса в стране?

**Сформулируем нулевую и альтернативные гипотезы**

*Нулевая гипотеза*:  Оценка резидента($μ_1$) такая же либо меньше, чем у не резидента($μ_2$):


$$ H_0 : μ_1=<μ_2$$

*Альтернативная гипотеза*:  Оценка резидента ($μ_1$) больше, чем оценка не резидента ($μ_2$):

$$ H_1 : μ_1>μ_2$$



**Проверка на нормальность**

С помощью теста Шапиро-Уилка проверим, распределён ли признак нормально. Напомним гипотезы теста Шапиро-Уилка:

*Нулевая гипотеза* ($H_0$): распределение данные является нормальным.

*Альтернативная гипотеза* ($H_1$): распределение данных отлично от нормального.

In [ ]:
# распределение оценок 
rs_r1 = hotels[mask_r_1]['reviewer_score']
rs_r0 =hotels[mask_r_0]['reviewer_score']

# проводим тест Шапиро-Уилка
print('Для резидентов:')
result = stats.shapiro(rs_r1)
decision_normality(result[1])

print()
print('Для не резидентов:')
result = stats.shapiro(rs_r0)
decision_normality(result[1])

**Выберем подходящий статистический тест**

Для выбора нужного теста воспользуемся алгоритмом выбора теста. Для этого ответим на следующие вопросы:
* Какой тип у признака? — Количественный.
* Сколько сравниваемых групп? — Две.
* Группы зависимы? — Нет.
* Признак распределён по нормальному закону? — Нет.

Чтобы проверить нашу гипотезу, можно использовать U-критерий Манна — Уитни для сравнения распределений на основе рангов.

**Проведём тест**

In [ ]:
# проводим тест
_, p = stats.mannwhitneyu(rs_r1,rs_r0, alternative='greater')
decision_hypothesis(p)

<span style="color:#59afe1">

**Вывод**

Статистический тест показал, что существует статистическая разница между оценкой отеля от резидента и нет.  
Оставляем данный признак для обучения модели.

In [ ]:
# добавим признак resident в список list_features
list_features.append('resident')

<span style="color:#59afe1"> 

### Создадим признак percentage_of_reviews <a id='p54'>

In [ ]:
# percentage_of_reviews = total_number_of_reviews/(total_number_of_reviews+additional_number_of_scoring)
hotels['percentage_of_reviews'] = hotels.apply(lambda x: x['total_number_of_reviews']/(x['additional_number_of_scoring']+x['total_number_of_reviews']),axis=1)

In [ ]:
# проанализируем корреляционную связь полученного признака и reviewer_score
# соберем матрицу корреляции для выбранных признаков
df_corr_pr = hotels[['percentage_of_reviews','reviewer_score']].corr()
df_corr_pr

<span style="color:#59afe1">

**Вывод**

Анализ показал очень слабую корреляционную связь.   
Данный признак не будем добавлять к списку признаков для обучения модели.

<span style="color:#59afe1"> 

### Обработка признака *review_date* <a id='p55'>

In [ ]:
# из признака review_date создадим признак review_year и month
hotels['review_year'] = pd.to_datetime(hotels['review_date']).dt.year
hotels['review_month'] = pd.to_datetime(hotels['review_date']).dt.month
# посмотрим на результат
hotels[['review_date','review_year','review_month']].info()

Проанализируем *reviewer_score* от года в котором был оставлен отзыв

In [ ]:
fig_4 = go.Figure()

# настроим оформление рабочей поверхности
fig_4.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от года рецензии', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_4.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_4.update_yaxes(
    type='category',
    categoryorder ='mean descending',
)

# Добавим диаграммы на рабочую поверхность
for mask in sorted(hotels['review_year'].unique()):
    fig_4.add_trace(go.Box(
        y=hotels[hotels['review_year'] == mask]['review_year'],
        x=hotels[hotels['review_year'] == mask]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=str(mask)
    ))


# расположим коробчатые графики горизонтально
fig_4.update_traces(
    orientation='h'
)
fig_4.show()

Из анализа графика можно сказать, что оценки отеля отличаются в зависимости от года рецензии, но эти отличия не выгладят статистически значимыми.  
Проведем для представленных распределений статистический тест.

Оценка рецензента зависит от года рецензии?

**Сформулируем нулевую и альтернативные гипотезы**

*Нулевая гипотеза*: Оценка рецензента не зависит от года рецензии:

$$ H_0 : μ_1 = μ_2 = μ_3$$

*Альтернативная гипотеза*: Оценка рецензента зависит от года рецензии.

$$ H_1 : μ_1 \neq μ_2 \neq μ_{3}$$



**Проверка на нормальность**

С помощью теста Шапиро-Уилка проверим, распределён ли признак нормально.

In [ ]:
# для сокращения длины кода осуществим проверку с помощью цикла для этого:
# проводим тест Шапиро-Уилка
for mask in sorted(hotels['review_year'].unique()):
    print('Распределение оценок рецензентов в',str(mask)+' году:')
    result = stats.shapiro(hotels[hotels['review_year']==mask]['reviewer_score'])
    decision_normality(result[1])
    print()
    
print('Количество сравниваемых групп:',len(hotels['review_year'].unique()))

**Выберем подходящий статистический тест**

Для выбора нужного теста воспользуемся алгоритмом выбора теста. Для этого ответим на следующие вопросы:
* Какой тип у исследуемого признака? — Количественный.
* Сколько сравниваемых групп? — Три.
* Группы зависимы? — Нет.
* Признак распределён по нормальному закону? — Нет. 

Для проверки нашей гипотезы можно использовать критерий Краскела — Уоллиса.

**Проведём тест**

In [ ]:
# соберем список из тестируемых зависимостей
list_dframes_y =[]
# извлечем их из словаря dict_var, полученного на этапе ранее
for mask in hotels['review_year'].unique():
    list_dframes_y.append(hotels[hotels['review_year']==mask]['reviewer_score'])
# проводим тест
_, p = stats.kruskal(*list_dframes_y)
decision_hypothesis(p)

<span style="color:#59afe1">

**Вывод**

Статистический тест показал, что оценка рецензента зависит от года рецензии.  
Оставляем данный признак для обучения модели.

In [ ]:
# добавим признак review_year в список list_features
list_features.append('review_year')

Проанализируем *reviewer_score* от месяца в котором был оставлен отзыв

In [ ]:
fig_5 = go.Figure()

# настроим оформление рабочей поверхности
fig_5.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от месяца рецензии', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_5.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_5.update_yaxes(
    type='category',
    categoryorder ='mean descending',
)

# Добавим диаграммы на рабочую поверхность
for mask in sorted(hotels['review_month'].unique()):
    fig_5.add_trace(go.Box(
        y=hotels[hotels['review_month'] == mask]['review_month'],
        x=hotels[hotels['review_month'] == mask]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=str(mask)
    ))


# расположим коробчатые графики горизонтально
fig_5.update_traces(
    orientation='h'
)
fig_5.show()

Из анализа графика можно сказать, что оценки отеля отличаются в зависимости от месяца рецензии.  
Проведем для представленных распределений статистический тест.

Оценка рецензента зависит от месяца рецензии?

**Сформулируем нулевую и альтернативные гипотезы**

*Нулевая гипотеза*: Оценка рецензента не зависит от месяца рецензии:

$$ H_0 : μ_1 = μ_2 = μ_3 = μ_4 = μ_5= μ_6 = μ_7 = μ_8 = μ_9 = μ_{10} = μ_{11} = μ_{12}$$

*Альтернативная гипотеза*: Оценка рецензента зависит от месяца рецензии.

$$ H_1 : μ_1 \neq μ_2  \neq μ_{3} \neq μ_{4} \neq μ_{5} \neq μ_{6} \neq μ_{7} \neq μ_{8} \neq μ_{9} \neq μ_{10} \neq μ_{11} \neq μ_{12}$$



**Проверка на нормальность**

С помощью теста Шапиро-Уилка проверим, распределён ли признак нормально.

In [ ]:
# для сокращения длины кода осуществим проверку с помощью цикла для этого:
# проводим тест Шапиро-Уилка
for mask in sorted(hotels['review_month'].unique()):
    print('Распределение оценок рецензентов в',str(mask)+' году:')
    result = stats.shapiro(hotels[hotels['review_month']==mask]['reviewer_score'])
    decision_normality(result[1])
    print()
    
print('Количество сравниваемых групп:',len(hotels['review_month'].unique()))

**Выберем подходящий статистический тест**

Для выбора нужного теста воспользуемся алгоритмом выбора теста. Для этого ответим на следующие вопросы:
* Какой тип у исследуемого признака? — Количественный.
* Сколько сравниваемых групп? — двенадцать.
* Группы зависимы? — Нет.
* Признак распределён по нормальному закону? — Нет. 

Для проверки нашей гипотезы можно использовать критерий Краскела — Уоллиса.

**Проведём тест**

In [ ]:
# соберем список из тестируемых зависимостей
list_dframes_m =[]
# извлечем их из словаря dict_var, полученного на этапе ранее
for mask in hotels['review_month'].unique():
    list_dframes_m.append(hotels[hotels['review_month']==mask]['reviewer_score'])
# проводим тест
_, p = stats.kruskal(*list_dframes_m)
decision_hypothesis(p)

<span style="color:#59afe1">

**Вывод**

Статистический тест показал, что оценка рецензента зависит от месяца рецензии.  
Оставляем данный признак для обучения модели.

In [ ]:
# добавим признак review_month в список list_features
list_features.append('review_month')

<span style="color:#59afe1"> 

### Обработка признака *hotel_name* <a id='p56'>

Проанализируем имена отелей с самыми высокими оценками рецензентов

In [ ]:
# выберем отели с оценкой рецензента больше 8.0
mask_top_s = hotels['reviewer_score'] >= 8.0
# разбиваем имена отелей на отдельные слова и заполняем ими список
list_g_word =[]
for  i in list(hotels[mask_top_s]['hotel_name'].value_counts().index):
    list_g_word+= i.split()
# посчитаем количество повторяющихся слов в с помощью библиотеки Counter
count_good_words = Counter(list_g_word)
# взглянем на полученный словарь
df_gw = pd.DataFrame(count_good_words.items(),columns=['Word','count'])
df_gw.sort_values(by='count', ascending=False)

Проанализируем имена отелей с самыми низкими оценками рецензентов

In [ ]:
# выберем отели с оценкой рецензента меньше 5.0
mask_worst_s = hotels['reviewer_score'] <= 5.0
# разбиваем имена отелей на отдельные слова и заполняем ими список
list_b_word =[]
for  i in list(hotels[mask_worst_s]['hotel_name'].value_counts().index):
    list_b_word+= i.split()
# посчитаем количество повторяющихся слов в с помощью библиотеки Counter
count_bad_words = Counter(list_b_word)
# взглянем на полученный словарь
df_bw = pd.DataFrame(count_bad_words.items(),columns=['Word','count'])
df_bw.sort_values(by='count', ascending=False)

Обратим внимание, что в каждом списке есть одинаковые слова.  
Причем они имеют практически одинаковый "вес" в списках.  
Поэтому, удалим из списков повторяющиеся значения.

Удалим повторяющие значения из списка "хороших" слов

In [ ]:
# список слов, полученных удалением из множества "хороших" слов множества "плохих"
list_good_word = list(set(df_gw['Word'])-set(df_bw['Word']))
len(list_good_word)

Удалим повторяющие значения из списка "плохих" слов

In [ ]:
# список слов, полученных удалением из множества "плохих" слов множества "хороших"
list_bad_word = list(set(df_bw['Word'])-set(df_gw['Word']))
len(list_bad_word)

список "плохих" слов пуст -создать соответствующий признак не получится

Создадим бинарные признак *good_words*

In [ ]:
# объявим функцию которую будет проверять строку на наличие в ней слов из списка
# для этого воспользуемся множествами
# проверим множество слов из строки на пересечение с множеством слов из списка
def intersection_sets(string,list_words):
    set_srt = set(string.split())               # множество слов из строки
    set_list = set(list_words)                  # множество слов из списка
    intersection = set_srt.isdisjoint(set_list)     # если есть пересечение множеств set_srt и set_list то длина set_intersection больше 0
    return not intersection                         # выведем ответ на вопрос есть ли пересечение во множествах в виде булевой переменной

In [ ]:
# создадим признак good_words
hotels['good_words'] = hotels['hotel_name'].apply(lambda x: 1 if intersection_sets(x,list_good_word) else 0)

In [ ]:
# посмотрим на полученный результат
hotels['good_words'].value_counts()

Проанализируем связь признака *good_words* с *reviewer_score*

In [ ]:
fig_6 = go.Figure()
# пропишем условия выбора данных
mask_g_1 = hotels['good_words'] == 1
mask_g_0 = hotels['good_words'] == 0

# сформируем список условий и имен
list_gw = [[mask_g_1,'good words'],[mask_g_0,'no good words']]

# настроим оформление рабочей поверхности
fig_6.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от наличия в имени  отеля "хороших" слов', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_6.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)


# Добавим диаграммы на рабочую поверхность
for mask in list_gw:
    fig_6.add_trace(go.Box(
        y=hotels[mask[0]]['good_words'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_6.update_traces(
    orientation='h'
)
fig_6.show()

<span style="color:#59afe1">

**Вывод**

Наличие корреляционной положительной связи очевидно.

In [ ]:
# добавим признак good_words в список list_features
list_features.append('good_words')

<span style="color:#59afe1"> 

### Обработка признака *reviewer_nationality* <a id='p57'>

Проанализируем признак *reviewer_nationality* на уникальные значения

In [ ]:
# проанализируем уникальные значения признака salary_currency
print(hotels['reviewer_nationality'].value_counts(normalize=True).head(20))
# посчитаем какой вес имеют ТОП 4 уникальных значений
print('В этом списке первые 49 значений составляют',round(100*hotels['reviewer_nationality'].value_counts(normalize=True).head(49).sum()),
      '% от',hotels['reviewer_nationality'].nunique(),'уникальных значений')

<span style="color:#59afe1"> 

О чем говорит анализ признака *reviewer_nationality*:
 - 176 уникальных значений, по отдельности являются не информативными;
 - значения, которые составляют 95% от всех уникальных можно выделить в 49 категорий.

Вывод: Признак *reviewer_nationality* требует преобразования. Разобьем признак на 50 категории:
1. Первые 49 категорий, это те, что вошли в ТОП 4 уникальных значений;
2. Все остальные поместим в категорию 'other'

In [ ]:
# получим список ТОП 49 уникальных значений
list_rn = hotels['reviewer_nationality'].value_counts(normalize=True).head(49).index

# Заполним признак 'reviewer_nationality' в соответствии с выбранной логикой
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x if x in list_rn else 'other')

Проанализируем распределение оценок рецензента в зависимости от национальности рецензента

In [ ]:
fig_7 = go.Figure()

# настроим оформление рабочей поверхности
fig_7.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от национальности', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 1600,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_7.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_7.update_yaxes(
    type='category',
    categoryorder ='mean descending',
)

# Добавим диаграммы на рабочую поверхность
for mask in sorted(hotels['reviewer_nationality'].unique()):
    fig_7.add_trace(go.Box(
        y=hotels[hotels['reviewer_nationality'] == mask]['reviewer_nationality'],
        x=hotels[hotels['reviewer_nationality'] == mask]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=str(mask)
    ))


# расположим коробчатые графики горизонтально
fig_7.update_traces(
    orientation='h'
)
fig_7.show()

Наличие статистически значимой связи очевидно. Для обучения модели кодируем признак reviewer_nationality.  
В нашем кодирование номер будет означать более высокие места в ТОП стран по среднему значению оценки рецензента.    
Первое место имеет самое большое значение кода.

In [ ]:
# составим список ТОП наций по среднему значению оценки рецензента
list_top_nationality = list(hotels.groupby('reviewer_nationality')['reviewer_score'].mean().sort_values().index)

# составим карту кодирования (Словарь со значениями)
dict_map_n ={}
for i in range(0,len(list_top_nationality)):
    dict_map_n[list_top_nationality[i]]=i+1

# создаем объект OrdinalEncoder, col - имя столбца, mapping - словарь с описанием кодировки
ord_encoder_n = ce.OrdinalEncoder(mapping=[{
	'col': 'reviewer_nationality',
	'mapping': dict_map_n
}])
# применяем трансформацию к столбцу
hotels['nationality_cod'] = ord_encoder_n.fit_transform(hotels[['reviewer_nationality']])
#  проверим корректность замены
print(hotels['nationality_cod'].value_counts().head(5),hotels['reviewer_nationality'].value_counts().head(5))

In [ ]:
# добавим признак nationality_cod в список list_features
list_features.append('nationality_cod')

<span style="color:#59afe1"> 

### Обработка признаков *positive_review* и *negative_review* <a id='p58'>

Составим ТОП friendly слов из отзывов, после которых рецензент оставил наибольшую оценку

In [ ]:
# заполним список friendly_word словами из positive_review
list_f_word = []
for  i in list(hotels[mask_top_s]['positive_review'].value_counts().index):
    list_f_word+= i.split()
# посчитаем количество повторяющихся слов в с помощью библиотеки Counter
count_friendly_words = Counter(list_f_word)
# взглянем на полученный словарь
df_fw = pd.DataFrame(count_friendly_words.items(),columns=['Word','count'])
df_fw.sort_values(by='count', ascending=False)

Составим ТОП not friendly слов из отзывов, после которых рецензент оставил наибольшую оценку

In [ ]:
# заполним список словами из полученной строки
list_nf_word = []
for  i in list(hotels[mask_worst_s]['negative_review'].value_counts().index):
    list_nf_word+= i.split()
# посчитаем количество повторяющихся слов в с помощью библиотеки Counter
count_nfriendly_words = Counter(list_nf_word)
# взглянем на полученный словарь
df_nfw = pd.DataFrame(count_nfriendly_words.items(),columns=['Word','count'])
df_nfw.sort_values(by='count', ascending=False)

Обратим внимание, что в каждом списке есть одинаковые слова.  
Причем они имеют практически одинаковый "вес" в списках.  
Поэтому, удалим из списков повторяющиеся значения.

удалим повторяющиеся значения из friendly списка

In [ ]:
# множество friendly слов, полученных удалением из множества "friendly" слов множества "not friendly"
set_friendly_word = set(df_fw['Word'])-set(df_nfw['Word'])
len(set_friendly_word)

удалим повторяющиеся значения из not friendly списка

In [ ]:
# множество not friendly слов, полученных удалением из множества "not friendly" слов множества "friendly"
set_nfriendly_word = set(df_nfw['Word'])-set(df_fw['Word'])
len(set_nfriendly_word)

создадим бинарные признаки *friendly_words* и *no_friendly_words*

In [ ]:
# создадим признак friendly_words
hotels['friendly_words'] = hotels['positive_review'].apply(lambda x: 1 if intersection_sets(x,set_friendly_word) else 0)
# посмотрим на полученный результат
hotels['friendly_words'].value_counts()

In [ ]:
# создадим признак *no_friendly_words*
hotels['no_friendly_words'] = hotels['negative_review'].apply(lambda x: 1 if intersection_sets(x,set_nfriendly_word) else 0)
# посмотрим на полученный результат
hotels['no_friendly_words'].value_counts()

Проанализируем связь полученных признаков с *reviewer_score*

In [ ]:
fig_fw = go.Figure()
# пропишем условия выбора данных
mask_f_1 = hotels['friendly_words'] == 1
mask_f_0 = hotels['friendly_words'] == 0

# сформируем список условий и имен
list_fw = [[mask_f_1,'friendly words'],[mask_f_0,'usual words']]

# настроим оформление рабочей поверхности
fig_fw.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от наличия в отзыве рецензента "friendly" слов', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_fw.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)


# Добавим диаграммы на рабочую поверхность
for mask in list_fw:
    fig_fw.add_trace(go.Box(
        y=hotels[mask[0]]['friendly_words'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_fw.update_traces(
    orientation='h'
)
fig_fw.show()

In [ ]:
fig_nfw = go.Figure()
# пропишем условия выбора данных
mask_nf_1 = hotels['no_friendly_words'] == 1
mask_nf_0 = hotels['no_friendly_words'] == 0

# сформируем список условий и имен
list_nfw = [[mask_nf_1,'no friendly words'],[mask_nf_0,'usual words']]

# настроим оформление рабочей поверхности
fig_nfw.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от наличия в отзыве рецензента "not friendly" слов', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_nfw.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)


# Добавим диаграммы на рабочую поверхность
for mask in list_nfw:
    fig_nfw.add_trace(go.Box(
        y=hotels[mask[0]]['no_friendly_words'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_nfw.update_traces(
    orientation='h'
)
fig_nfw.show()

<span style="color:#59afe1">

**Вывод**

Наличие корреляционной положительной связи в случае с признаком *friendly_words*   
и корреляционной отрицательной связи в случае с признаком *no_friendly_words* очевидно.

In [ ]:
# добавим признаки no_friendly_words и friendly_words в список list_features
list_features.extend(['friendly_words','no_friendly_words'])

создадим признак *count_positive_reviews*, отражающий общее число позитивный отзывов для отеля.  
Анализ признака positive_review показывает, что в нем присутствуют следующие значения:  
'No Positive', ' Nothing', ' Nothing ', ' nothing', ' Not much', ' ', которые отражают   
отсутствие позитивного отзыва. Общее количество позитивных отзывов будет равно:  
(общее число позитивных отзывов) = (все отзывы) - (отзывы из списка выше)  

In [ ]:
# выберем из positive_review позитивные отзывы
list_n_positive_review =['No Positive', ' Nothing', ' Nothing ', ' nothing', ' Not much', ' ',' Almost nothing']   # список не позитивных отзывов
mask_p_reviews = hotels['positive_review'].apply(lambda x: x not in list_n_positive_review)
# создадим Series с именем отеля и количеством позитивных отзывов рецензентов
s_cp = pd.Series(hotels[mask_p_reviews].groupby(by='hotel_name')['positive_review'].count(), name='count_positive_reviews')
# посчитаем общее количество отзывов для каждого отеля и запишем результат в новый Series
s_cpg = pd.Series(hotels.groupby(by='hotel_name')['positive_review'].count(), name='count_reviews')

In [ ]:
# Проверим что число отелей в s_cp совпадает с общим числом отелей
s_cp.count()==hotels['hotel_name'].value_counts().count()

In [ ]:
# объединим полученные Series в Data Frame
df_pr =pd.concat([s_cp,s_cpg],axis=1)
# добавим признак percentage_positive_reviews, отражающий долю позитивных отзывов от общего количества positive_reviews
df_pr['percentage_positive_reviews'] = df_pr['count_positive_reviews']/df_pr['count_reviews']
# посмотрим на результат
df_pr

In [ ]:
# "перенесем" признак count_positive_reviews в основной DataFrame
hotels['count_positive_reviews'] = hotels['hotel_name'].apply(lambda x: df_pr.loc[x][0])
# "перенесем" признак percentage_positive_reviews в основной DataFrame
hotels['percentage_positive_reviews'] = hotels['hotel_name'].apply(lambda x: df_pr.loc[x][2])
# посмотрим на результат
hotels[['hotel_name','count_positive_reviews','percentage_positive_reviews']].head(10)

проанализируем корреляционную связь признаков *count_positive_reviews*, percentage_positive_reviews* и *reviewer_score*

In [ ]:
# проанализируем корреляционную связь полученного признака и reviewer_score
# соберем матрицу корреляции для выбранных признаков
df_corr_cpr = hotels[['count_positive_reviews','percentage_positive_reviews','reviewer_score']].corr()
df_corr_cpr

<span style="color:#59afe1">

**Вывод**

Анализ показал очень слабую корреляционную связь признака *count_positive_reviews* и *reviewer_score*.  
Данный признак не будем добавлять к списку признаков для обучения модели.  
При этом, признак *percentage_positive_reviews* обладает слабой положительной корреляционной связью с  
признаком *reviewer_score*. Добавим этот признак к обучению модели.

In [ ]:
# добавим признак percentage_positive_reviews в список list_features
list_features.append('percentage_positive_reviews')

<span style="color:#59afe1"> 

### Обработка признака *tags* <a id='p59'>

#### выделим признак бинарный признак *with a pet* отражающий наличие животных у посетителя

In [ ]:
# если посетитель с животными ставим 1
hotels['with_a_pet'] =hotels['tags'].apply(lambda x: 1 if 'With a pet' in x else 0 )
# посмотрим на результат
print(hotels[['tags','with_a_pet']].head(5))
print(hotels[hotels['with_a_pet'] == 1][['tags','with_a_pet']].head(5))


Проанализируем связь полученного признака с *reviewer_score*

In [ ]:
fig_pet = go.Figure()
# пропишем условия выбора данных
mask_p_1 = hotels['with_a_pet'] == 1
mask_p_0 = hotels['with_a_pet'] == 0

# сформируем список условий и имен
list_pet = [[mask_p_1,'with a pet'],[mask_p_0,'without pet']]

# настроим оформление рабочей поверхности
fig_pet.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от наличия у него животных', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_pet.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)


# Добавим диаграммы на рабочую поверхность
for mask in list_pet:
    fig_pet.add_trace(go.Box(
        y=hotels[mask[0]]['with_a_pet'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_pet.update_traces(
    orientation='h'
)
fig_pet.show()

Из анализа графика можно сказать, что оценка отеля отличается в зависимости от наличия животных у посетителя, но эти отличия не выгладят статистически значимыми.  
Проведем для представленных распределений статистический тест.

Как соотносятся оценки рецензентов в зависимости от наличия у них животных?

**Сформулируем нулевую и альтернативные гипотезы**

*Нулевая гипотеза*:  Оценка резидента без животных ($μ_1$) такая же либо меньше, чем у не резидента с животными ($μ_2$):


$$ H_0 : μ_1=<μ_2$$

*Альтернативная гипотеза*:  Оценка резидента без животных ($μ_1$) больше, чем оценка не резидента с животными ($μ_2$):

$$ H_1 : μ_1>μ_2$$

**Проверка на нормальность**

С помощью теста Шапиро-Уилка проверим, распределён ли признак нормально. Напомним гипотезы теста Шапиро-Уилка:

*Нулевая гипотеза* ($H_0$): распределение данные является нормальным.

*Альтернативная гипотеза* ($H_1$): распределение данных отлично от нормального.

In [ ]:
# распределение оценок 
rs_p1 = hotels[mask_p_1]['reviewer_score']
rs_p0 =hotels[mask_p_0]['reviewer_score']

# проводим тест Шапиро-Уилка
print('Для рецензентов с животными:')
result = stats.shapiro(rs_p1)
decision_normality(result[1])

print()
print('Для рецензентов без животных:')
result = stats.shapiro(rs_p0)
decision_normality(result[1])

**Выберем подходящий статистический тест**

Для выбора нужного теста воспользуемся алгоритмом выбора теста. Для этого ответим на следующие вопросы:
* Какой тип у признака? — Количественный.
* Сколько сравниваемых групп? — Две.
* Группы зависимы? — Нет.
* Признак распределён по нормальному закону? — Нет.

Чтобы проверить нашу гипотезу, можно использовать U-критерий Манна — Уитни для сравнения распределений на основе рангов.

**Проведём тест**

In [ ]:
# проводим тест
_, p = stats.mannwhitneyu(rs_p0,rs_p1, alternative='greater')
decision_hypothesis(p)

<span style="color:#59afe1">

**Вывод**

Статистический тест показал отсутствие статистически значимой разницы между оценкой отеля от рецензента с животным и без них.  
Признак *with_a_pet* не будет участвовать в обучении модели.

#### выдели признак *type_of_trip* отражающий характер поездки

In [ ]:
# 0, если отсутствует информация 
# 1, если Leisure trip
# 2, если Business trip
hotels['type_of_trip'] =hotels['tags'].apply(lambda x: 0 if 'trip' not in x else (1 if 'Leisure' in x else 2) )
# посмотрим на результат
print(hotels[hotels['type_of_trip'] == 0][['tags','type_of_trip']].head(5))
print(hotels[hotels['type_of_trip'] == 1][['tags','type_of_trip']].head(5))
print(hotels[hotels['type_of_trip'] == 2][['tags','type_of_trip']].head(5))

Проанализируем связь полученного признака с *reviewer_score*

In [ ]:
fig_trip = go.Figure()
# пропишем условия выбора данных
mask_trip_0 = hotels['type_of_trip'] == 0
mask_trip_1 = hotels['type_of_trip'] == 1
mask_trip_2 = hotels['type_of_trip'] == 2

# сформируем список условий и имен
list_trip = [[mask_trip_0,'without_info'],[mask_trip_1,'Leisure trip'],[mask_trip_2,'Business trip']]

# настроим оформление рабочей поверхности
fig_trip.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от характера поездки', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_trip.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_trip.update_yaxes(
    type='category',
    categoryorder ='mean ascending',
)


# Добавим диаграммы на рабочую поверхность
for mask in list_trip:
    fig_trip.add_trace(go.Box(
        y=hotels[mask[0]]['type_of_trip'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_trip.update_traces(
    orientation='h'
)
fig_trip.show()

<span style="color:#59afe1">

**Вывод**

Наличие статистически значимой связи очевидно.

In [ ]:
# добавим признак type_of_trip в список list_features
list_features.append('type_of_trip')

####  выдели признак *nights_to_stay* отражающий количество ночей на которые остался посетитель

In [ ]:
# NAN, при отсутствие информации. после нулевые значения заменим на среднее количество ночей.
# количество ночей, при наличие информации найдем с помощью регулярных выражений
hotels['nights_to_stay'] =hotels['tags'].apply(lambda x: np.nan if 'Stayed' not in x else int(re.findall('Stayed \d*',x)[0][7:]) )
# посмотрим на количество пропущенных значений
hotels['nights_to_stay'].isnull().value_counts()

In [ ]:
# отыщем среднее число ночей округленное до целого
mean_count_nights = round(hotels[hotels['nights_to_stay']!=0]['nights_to_stay'].mean())
# заменим пропуски в признаке nights_to_stay найденным значением
hotels['nights_to_stay'].fillna(mean_count_nights,inplace=True)
# посмотрим на результат
hotels['nights_to_stay'].isnull().value_counts()

Проанализируем корреляционную связь полученного признака с *reviewer_score*

In [ ]:
# соберем матрицу корреляции для выбранных признаков
df_corr_pr = hotels[['nights_to_stay','reviewer_score']].corr()
df_corr_pr

<span style="color:#59afe1">

**Вывод**

Корреляционный анализ показал отсутствие статистически значимой корреляционной связи между признаками.

####  выдели бинарный признак *view* отражающий опции номера посетителя (наличие вида)

In [ ]:
# если номер с видом ставим 1
hotels['view'] =hotels['tags'].apply(lambda x: 1 if 'View' in x else 0 )
# проанализируем результат на результат
hotels['view'].value_counts()


Проанализируем связь полученного признака с *reviewer_score*

In [ ]:
fig_view = go.Figure()
# пропишем условия выбора данных
mask_v_1 = hotels['view'] == 1
mask_v_0 = hotels['view'] == 0

# сформируем список условий и имен
list_view = [[mask_v_1,'with a view'],[mask_v_0,'without view']]

# настроим оформление рабочей поверхности
fig_view.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от наличия view номера', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_view.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)


# Добавим диаграммы на рабочую поверхность
for mask in list_view:
    fig_view.add_trace(go.Box(
        y=hotels[mask[0]]['view'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_view.update_traces(
    orientation='h'
)
fig_view.show()

<span style="color:#59afe1">

**Вывод**

Наличие статистически значимой связи очевидно.

In [ ]:
# добавим признак view в список list_features
list_features.append('view')

####   выдели категориальный признак *type_of_premium_room* отражающий тип номера повышенной комфортности

In [ ]:
# комнатой уровня премиум будем считать : Superior, Deluxe, Suite, Luxury
# 0, если комната не из списка premium
# 1, если Superior
# 2, если Deluxe
# 3, если Suite
# 4, если Luxury

hotels['type_of_premium_room'] =hotels['tags'].apply(lambda x: 4 if 'Luxury' in x else (3 if 'Suite' in x else (2 if 'Deluxe' in x else (1 if 'Superior' in x else 0))) )
# посмотрим на результат
hotels['type_of_premium_room'].value_counts()

Проанализируем связь полученного признака с *reviewer_score*

In [ ]:
fig_room = go.Figure()
# пропишем условия выбора данных
mask_room_0 = hotels['type_of_premium_room'] == 0
mask_room_1 = hotels['type_of_premium_room'] == 1
mask_room_2 = hotels['type_of_premium_room'] == 2
mask_room_3 = hotels['type_of_premium_room'] == 3
mask_room_4 = hotels['type_of_premium_room'] == 4

# сформируем список условий и имен
list_room = [[mask_room_0,'usual room'],[mask_room_1,'Superior room'],[mask_room_2,'Deluxe room'],[mask_room_3,'Suite room'],[mask_room_4,'Luxury room']]

# настроим оформление рабочей поверхности
fig_room.update_layout(
    title ={
        'text':'Распределение оценок рецензента в зависимости от типа комнаты', # Имя рабочей плоскости
        'font':{'size':35,'family':"Times New Roman"}, # размер и стиль написания имени рабочей плоскости
        'x':0.5, # Смешение имени по оси "x" на половину рабочей плоскости
        },
    height = 800,# Высота рабочей плоскости
    width = 1600, # Ширина рабочей плоскости
    template='simple_white', # зададим тему оформления для рабочей поверхности 
    )
fig_room.update_xaxes(
    title = 'Оценка рецензента' # подпишем ось абсцисс
)
fig_room.update_yaxes(
    type='category',
    categoryorder ='mean ascending',
)


# Добавим диаграммы на рабочую поверхность
for mask in list_room:
    fig_room.add_trace(go.Box(
        y=hotels[mask[0]]['type_of_premium_room'],
        x=hotels[mask[0]]['reviewer_score'],
        boxmean=True,
        hoverinfo='x',
        name=mask[1])
                    )

# расположим коробчатые графики горизонтально
fig_room.update_traces(
    orientation='h'
)
fig_room.show()

<span style="color:#59afe1">

**Вывод**

Наличие статистически значимой связи очевидно.

In [ ]:
# добавим признак type_of_premium_room в список list_features
list_features.append('type_of_premium_room')

<span style="color:#59afe1"> 

### Обработка признака *lat* и *lng* <a id='p510'>

In [ ]:
hotels[['lat','lng']].isnull().value_counts()

кака показывает анализ в нашей выборке есть отели у которых отсутствует информация   
о их географическом положение. Отыщем эти отели.

In [ ]:
# получим список отелей с пропусками гео данных 
list_without_geo = list(hotels[hotels['lat'].isnull()]['hotel_name'].unique())
list_without_geo

In [ ]:
# создадим Data frame c отелями из списка list_without_geo и их геоданными
df_hotel_w_geo =pd.DataFrame(
    index =list_without_geo,
    columns = ['lat','lng']
)
# заполним геоданных выбранных отелей с помощью библиотеки geopy.geocoders
# учтем тот факт, что некоторые имена отелей не сможет отыскать библиотека
# на этот случай используем конструкцию try-except
# найдем список отелей с "не точными" именами
list_not_correct_name = []
for hotel_name in df_hotel_w_geo.index:
    try:
        loc = Nominatim(user_agent='hotel_name')
        getloc = loc.geocode(hotel_name)
        df_hotel_w_geo['lat'][hotel_name] = getloc.latitude
        df_hotel_w_geo['lng'][hotel_name] = getloc.longitude
    except AttributeError:
        list_not_correct_name.append(hotel_name)
# посмотрим на результат
df_hotel_w_geo

In [ ]:
# посмотрим на список отелей с не корректными именами
list_not_correct_name

In [ ]:
# в ручную отыщем для них адреса
list_correct_address =['Josefstädter Str. 10-12, 1080 Wien, Австрия',
                       '4 Rue de la Pépinière, 75008 Paris, Франция',
                       'Sieveringer Str. 4, 1190 Wien, Австрия',
                       'Grünentorgasse 30, 1090 Wien, Австрия',
                       'Währinger Str. 12, 1090 Wien, Австрия',
                       '20 rue de la Gaité, 75014 PARIS, France ']

In [ ]:
# создаем пустой словарь
dict_correct_address ={}
# составим словарь (имя отеля - корректный адрес)
for i in range(len(list_not_correct_name)):
    dict_correct_address[list_not_correct_name[i]] = list_correct_address[i]

dict_correct_address

In [ ]:
for hotel,address in dict_correct_address.items():
    print(hotel)
    print(address)
    

In [ ]:
# повторно заполним df_hotel_w_geo
list_not_correct_name = []
for hotel_name,address in dict_correct_address.items():
    try:
        loc = Nominatim(user_agent='hotel_name')
        getloc = loc.geocode(address)
        df_hotel_w_geo['lat'][hotel_name] = getloc.latitude
        df_hotel_w_geo['lng'][hotel_name] = getloc.longitude
    except AttributeError:
        list_not_correct_name.append(hotel_name)
# посмотрим на результат
df_hotel_w_geo

In [ ]:
# проверяем что список list_not_correct_name пуст
list_not_correct_name

In [ ]:
# заполним недостающими данными признак lat основной Data Frame
hotels['lat'] = hotels.apply(lambda x: df_hotel_w_geo['lat'][x['hotel_name']] if x['hotel_name'] in df_hotel_w_geo.index else x['lat'],axis=1)
# проверим результат
hotels['lat'].notnull().value_counts()

In [ ]:
# заполним недостающими данными признак lng основной Data Frame
hotels['lng'] = hotels.apply(lambda x: df_hotel_w_geo['lng'][x['hotel_name']] if x['hotel_name'] in df_hotel_w_geo.index else x['lng'],axis=1)
# проверим результат
hotels['lng'].notnull().value_counts()

создадим признаки с координатами центра города 

In [ ]:
# получим список городов
list_cities = hotels['city'].value_counts().index
list_cities

In [ ]:
# создадим Data frame c городами из списка list_cities и их геоданными
df_cities_geo =pd.DataFrame(
    index =list_cities,
    columns = ['lat','lng']
)
# заполним геоданных выбранных отелей с помощью библиотеки geopy.geocoders
for city in df_cities_geo.index:
        loc = Nominatim(user_agent='hotel_name')
        getloc = loc.geocode(city)
        df_cities_geo['lat'][city] = getloc.latitude
        df_cities_geo['lng'][city] = getloc.longitude

# посмотрим на результат
df_cities_geo

In [ ]:
# в основном Data Frame создадим признаки с координатами центра города
hotels['city_center_lat'] = hotels['city'].apply(lambda x: df_cities_geo['lat'][x])
hotels['city_center_lng'] = hotels['city'].apply(lambda x: df_cities_geo['lng'][x])
# проверим результат
hotels[['city','city_center_lat','city_center_lng']].head(5)

создадим признак *distance_to_center*≤ показывающий расстояние от центра города до отеля

In [ ]:
# для создания признака используем библиотеку geopy.distance
hotels['distance_to_center'] = hotels.apply(lambda x: round(geodesic((x['city_center_lat'],x['city_center_lng']),(x['lat'],x['lng'])).kilometers,3),axis=1)
# проверим результат
hotels[['hotel_name','city','distance_to_center']]

Проанализируем корреляционную связь полученного признака с *reviewer_score*

In [ ]:
# соберем матрицу корреляции для выбранных признаков
df_corr_pr = hotels[['distance_to_center','reviewer_score']].corr()
df_corr_pr

<span style="color:#59afe1">

**Вывод**

Корреляционный анализ показал отсутствие статистически значимой корреляционной связи между признаками.

<span style="color:#59afe1"> 

### Выбор признаков для обучения модели <a id='p511'>

После проведения разведывательного анализа был получен список list_features с признаками для обучения модели

In [ ]:
# добавим в список list_features целевой признак reviewer_score
list_features.append('reviewer_score')

In [ ]:
# взглянем на полученный список list_features
print(list_features)
print('количество признаков:', len(list_features))

In [ ]:
# соберем Data Frame с полученным списком признаков
df_for_model = hotels[list_features]
# посмотрим на полученный Data Frame
df_for_model.head(5)

In [ ]:
df_for_model.info()

<span style="color:#59afe1"> 

## ПРОВЕРКА ОБУЧАЮЩИХ ПРИЗНАКОВ<a id='p6'>

In [ ]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х — данные с информацией об отелях, у — целевая переменная (оценка рецензента)  
X = df_for_model.drop(['reviewer_score'], axis = 1)  
y = df_for_model['reviewer_score']  
      
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" — для тестирования.  
# Для тестирования мы будем использовать 25 % от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)  

# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)

In [ ]:
# Используем обученную модель для предсказания оценки рецензента в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test) и смотрим, насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений в отношении фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(regr.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

<span style="color:#59afe1"> 

## ПОДГОТОВКА ТЕСТОВЫХ ДАННЫХ <a id='p7'>

<span style="color:#59afe1"> 

### загрузка данных <a id='p71'>

In [ ]:
# загружаем данные для тестирования модели
test_data  = pd.read_csv('data/hotels_test.csv')
test_data.info()

In [ ]:
# загружаем данные для заполнения результатами предсказаний
sample_submission  = pd.read_csv('data/submission.csv')
sample_submission.sample(2)

<span style="color:#59afe1"> 

### создадим признак *country_cod* <a id='p72'>

In [ ]:
# создадим признак страны отеля
test_data['country']= test_data['hotel_address'].str.findall('\w+').str.get(-1)
# заменим значение Kingdom на United Kingdom
test_data['country']=test_data['country'].apply(lambda x: 'United Kingdom' if x == 'Kingdom' else x)
test_data['country'].value_counts()

In [ ]:
# создадим признак города отеля
test_data['city']= test_data['hotel_address'].str.findall('\w+').str.get(-2)
# заменим значение United на London
test_data['city']=test_data['city'].apply(lambda x: 'London' if x == 'United' else x)
test_data['city'].value_counts()

In [ ]:
# закодируем признак country с помощью объекта OrdinalEncoder
# применяем трансформацию к столбцу
test_data['country_cod'] = ord_encoder_c.fit_transform(test_data[['country']])
#  проверим корректность замены
print(test_data['country'].value_counts(),test_data['country_cod'].value_counts())

<span style="color:#59afe1"> 

### создадим признак *resident* <a id='p73'>

In [ ]:
# значение в признаки 1 - посетитель резидент страны
# значение признака 0 - посетитель не резидент страны
test_data['resident'] = test_data.apply(lambda x: 1 if (x['country']==x['reviewer_nationality'][1:-1]) else 0,axis=1)
# проверим фрагмент таблицы
test_data[['country','reviewer_nationality','resident']].head(10)

#### создадим признаки *review_year* и *review_month*

In [ ]:
# из признака review_date создадим признак review_year и month
test_data['review_year'] = pd.to_datetime(test_data['review_date']).dt.year
test_data['review_month'] = pd.to_datetime(test_data['review_date']).dt.month
# посмотрим на результат
test_data[['review_date','review_year','review_month']].sample(5)

<span style="color:#59afe1"> 

### создадим признак *good_words* <a id='p74'>

In [ ]:
# создадим признак good_words
test_data['good_words'] = test_data['hotel_name'].apply(lambda x: 1 if intersection_sets(x,list_good_word) else 0)
# посмотрим на результат
test_data['good_words'].value_counts()

<span style="color:#59afe1"> 

### создадим признак *nationality_cod* <a id='p75'>

In [ ]:
# Выберем из признака 'reviewer_nationality' ТОП 50 категорий
test_data['reviewer_nationality'] = test_data['reviewer_nationality'].apply(lambda x: x if x in list_rn else 'other')

In [ ]:
# кодируем признак reviewer_nationality с помощью объекта ord_encoder_n
test_data['nationality_cod'] = ord_encoder_n.fit_transform(test_data[['reviewer_nationality']])
#  проверим корректность замены
print(test_data['nationality_cod'].value_counts().head(5),test_data['reviewer_nationality'].value_counts().head(5))

<span style="color:#59afe1"> 

### создадим признаки *friendly_words* и *no_friendly_words* <a id='p76'>

In [ ]:
# создадим признак friendly_words
test_data['friendly_words'] = test_data['positive_review'].apply(lambda x: 1 if intersection_sets(x,set_friendly_word) else 0)
# посмотрим на полученный результат
test_data['friendly_words'].value_counts()

In [ ]:
# создадим признак *no_friendly_words*
test_data['no_friendly_words'] = test_data['negative_review'].apply(lambda x: 1 if intersection_sets(x,set_nfriendly_word) else 0)
# посмотрим на полученный результат
test_data['no_friendly_words'].value_counts()

<span style="color:#59afe1"> 

### создадим признак *percentage_positive_reviews* <a id='p77'>

In [ ]:
# выберем из positive_review позитивные отзывы
list_n_positive_review =['No Positive', ' Nothing', ' Nothing ', ' nothing', ' Not much', ' ',' Almost nothing']   # список не позитивных отзывов
mask_p_reviews = test_data['positive_review'].apply(lambda x: x not in list_n_positive_review)
# создадим Series с именем отеля и количеством позитивных отзывов рецензентов
s_cp = pd.Series(test_data[mask_p_reviews].groupby(by='hotel_name')['positive_review'].count(), name='count_positive_reviews')
# посчитаем общее количество отзывов для каждого отеля и запишем результат в новый Series
s_cpg = pd.Series(test_data.groupby(by='hotel_name')['positive_review'].count(), name='count_reviews')

In [ ]:
# Проверим что число отелей в s_cp совпадает с общим числом отелей
s_cp.count()==test_data['hotel_name'].value_counts().count()

In [ ]:
# объединим полученные Series в Data Frame
df_pr =pd.concat([s_cp,s_cpg],axis=1)
# добавим признак percentage_positive_reviews, отражающий долю позитивных отзывов от общего количества positive_reviews
df_pr['percentage_positive_reviews'] = df_pr['count_positive_reviews']/df_pr['count_reviews']
# посмотрим на результат
df_pr

In [ ]:
# "перенесем" признак count_positive_reviews в основной DataFrame
test_data['count_positive_reviews'] = test_data['hotel_name'].apply(lambda x: df_pr.loc[x][0])
# "перенесем" признак percentage_positive_reviews в основной DataFrame
test_data['percentage_positive_reviews'] = test_data['hotel_name'].apply(lambda x: df_pr.loc[x][2])
# посмотрим на результат
test_data[['hotel_name','count_positive_reviews','percentage_positive_reviews']].head(10)

<span style="color:#59afe1"> 

### создадим признак *type_of_trip* <a id='p78'>

In [ ]:
# 0, если отсутствует информация 
# 1, если Leisure trip
# 2, если Business trip
test_data['type_of_trip'] =test_data['tags'].apply(lambda x: 0 if 'trip' not in x else (1 if 'Leisure' in x else 2) )
# посмотрим на результат
print(test_data[test_data['type_of_trip'] == 0][['tags','type_of_trip']].head(5))
print(test_data[test_data['type_of_trip'] == 1][['tags','type_of_trip']].head(5))
print(test_data[test_data['type_of_trip'] == 2][['tags','type_of_trip']].head(5))

<span style="color:#59afe1"> 

### создадим признак *view* <a id='p79'>

In [ ]:
# если номер с видом ставим 1
test_data['view'] =test_data['tags'].apply(lambda x: 1 if 'View' in x else 0 )
# проанализируем результат на результат
test_data['view'].value_counts()

<span style="color:#59afe1"> 

### создадим признак *type_of_premium_room* <a id='p710'>

In [ ]:
# комнатой уровня премиум будем считать : Superior, Deluxe, Suite, Luxury
# 0, если комната не из списка premium
# 1, если Superior
# 2, если Deluxe
# 3, если Suite
# 4, если Luxury

test_data['type_of_premium_room'] =test_data['tags'].apply(lambda x: 4 if 'Luxury' in x else (3 if 'Suite' in x else (2 if 'Deluxe' in x else (1 if 'Superior' in x else 0))) )
# посмотрим на результат
test_data['type_of_premium_room'].value_counts()

<span style="color:#59afe1"> 

### Результирующие test_data <a id='p711'>

In [ ]:
# посмотрим на полученный Data Frame
test_data.sample(2)

In [ ]:
# посмотрим на список признаков по которым обучается модель
list_features

In [ ]:
# соберем r_test_data с признаками из списка list_features исключив признак reviewer_score
r_test_data = test_data[list_features[:-1]]
# посмотрим на полученный Data Frame
r_test_data.head(5)

<span style="color:#59afe1"> 

## МАШИННОЕ ОБУЧЕНИЕ <a id='p8'>

In [ ]:
# на этапе машинного обучение проведем обучение модели на всех данных из датасета df_for_model  
# Х — данные с информацией об отелях, у — целевая переменная (рейтинги отелей)  
X_train = df_for_model.drop(['reviewer_score'], axis = 1)  
y_train = df_for_model['reviewer_score']

In [ ]:
# Создаём модель  
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

In [ ]:
# Используем обученную модель для предсказания оценки рецензента в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
predict_submission = model.predict(r_test_data)

In [ ]:
# запишем результат предсказаний в файл sample_submission
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('data/submission.csv', index=False)
sample_submission.head(10)